## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
mode_of_interest = None
include_test_users = False
use_imperial = False
sensed_algo_prefix = "cleaned"
use_imperial = True

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
#set the label-units just once!
if use_imperial:
    label_units = "Miles"
    short_label = "miles"
else:
    label_units = "Kilometers"
    short_label = "kms"
    
label_units_lower = label_units.lower()
distance_col = "distance_" + short_label

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_sensor_inference_data(year,
                                                                            month,
                                                                            program,
                                                                            include_test_users,
                                                                            sensed_algo_prefix)

## Generic Metrics

### Distribution of Mode_confirm attribute

In [ ]:
file_name='ntrips_sensed_mode%s' % file_suffix
plot_title_no_quality= "Number of trips for each primary mode\n(inferred by OpenPATH from phone sensors)"
try:
    labels_mc = expanded_ct['primary_mode'].value_counts(dropna=True).keys().tolist()
    values_mc = expanded_ct['primary_mode'].value_counts(dropna=True).tolist()    
    plot_title = plot_title_no_quality+"\n"+quality_text
    pie_chart_sensed_mode(plot_title,labels_mc,values_mc,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_mc, labels_mc), file_name, plot_title)
    print(expanded_ct['primary_mode'].value_counts(dropna=True))
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Mode choice for trips under 80% mark

In [ ]:
#determine 80th percentile
cutoff = expanded_ct.distance.quantile(0.8)
dist_threshold = expanded_ct[distance_col].quantile(0.8).round(1)
dist_threshold = str(dist_threshold)

plot_title_no_quality="Number of trips under " + dist_threshold + " " + label_units_lower + " for each primary mode"
plot_title_no_quality=plot_title_no_quality + "\n(inferred by OpenPATH from phone sensors)" 
plot_title_no_quality=plot_title_no_quality + "\n["+dist_threshold + " " + label_units_lower+" represents 80th percentile of trip length]"
file_name ='ntrips_under10miles_sensed_mode%s' % file_suffix

try:
    labels_d10 = expanded_ct.loc[(expanded_ct['distance'] <= cutoff)].primary_mode.value_counts(dropna=True).keys().tolist()
    values_d10 = expanded_ct.loc[(expanded_ct['distance'] <= cutoff)].primary_mode.value_counts(dropna=True).tolist()
    d10_quality_text = scaffolding.get_quality_text(expanded_ct, expanded_ct[expanded_ct['distance'] <= cutoff], "< " + dist_threshold + " " + label_units_lower, include_test_users)
    plot_title= plot_title_no_quality+"\n"+d10_quality_text
    pie_chart_sensed_mode(plot_title,labels_d10,values_d10,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_d10, labels_d10), file_name, plot_title)
    print(expanded_ct.loc[(expanded_ct['distance'] <= cutoff)].primary_mode.value_counts(dropna=True))
except:
    d10_df = expanded_ct.query("distance <= " + cutoff) if "distance" in expanded_ct.columns else expanded_ct
    debug_df.loc["Trips_less_than_80th_pct"] = scaffolding.trip_label_count("Mode_confirm", d10_df)
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Miles per chosen transport mode

In [ ]:
plot_title_no_quality = label_units + " for each primary mode\n(inferred by OpenPATH from phone sensors)"
plot_title = plot_title_no_quality + quality_text
file_name ='miles_sensed_mode%s' % file_suffix

try:
    dist = expanded_ct.groupby('primary_mode').agg({distance_col: ['sum', 'count' , 'mean']})
    dist.columns = ['Total ('+label_units_lower+')', 'Count', 'Average ('+label_units_lower+')']
    dist = dist.reset_index()
    dist = dist.sort_values(by=['Total ('+label_units_lower+')'], ascending=False)
    dist_dict = dict(zip(dist['primary_mode'], dist['Total ('+label_units_lower+')']))

    labels_m = []
    values_m = []

    for x, y in dist_dict.items():
        labels_m.append(x)
        values_m.append(y)
        
    pie_chart_sensed_mode(plot_title,labels_m,values_m,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)

except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Miles per chosen land transport mode

In [ ]:
plot_title_no_quality= label_units + " for each land-based primary mode\n(inferred by OpenPATH from phone sensors)"
plot_title = plot_title_no_quality + quality_text
file_name ='miles_sensed_mode_land%s' % file_suffix

try:
    dist = expanded_ct.groupby('primary_mode').agg({distance_col: ['sum', 'count' , 'mean']})
    dist.columns = ['Total ('+label_units_lower+')', 'Count', 'Average ('+label_units_lower+')']
    dist = dist.reset_index()
    dist =dist.sort_values(by=['Total ('+label_units_lower+')'], ascending=False)

    dist_dict = dict(zip(dist['primary_mode'], dist['Total ('+label_units_lower+')']))

    labels_m = []
    values_m = []

    for x, y in dist_dict.items():
        if x != "AIR_OR_HSR":
            labels_m.append(x)
            values_m.append(y)

    pie_chart_sensed_mode(plot_title,labels_m,values_m,file_name)
    alt_text = store_alt_text_pie(pd.DataFrame(values_m, labels_m), file_name, plot_title)

except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day¶

In [ ]:
plot_title_no_quality="Number of trips by day\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_day%s' % file_suffix

try:
    fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'
    
    plot_title= plot_title_no_quality+"\n"+quality_text

    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality="Number of trips by weekday\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_weekday%s' % file_suffix
try:
    fq_weekdays = expanded_ct.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    